### PRE-REQUISITS

In [39]:
import os 
import dotenv
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [5]:
from langchain_core.documents import Document
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [6]:
#  embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [29]:

qa_dataset = {

         "What is the largest planet in our solar system?":  "Jupiter",
         "Which planet is known as the Red Planet?": "Mars",
         "What is the name of the first human to walk on the moon?": "Neil Armstrong",

         "What does 'API' stand for?": "Application Programming Interface",
         "Which language is primarily used for Android app development in 2026?": "Kotlin",
         "What is a 'boolean' in programming?": "A data type that has one of two possible values: true or false.",

         "What is the main ingredient in traditional guacamole?": "Avocado",
         "What does the French term 'Sous-vide' mean?": "Under vacuum",
         "Which spice is known as the most expensive in the world?": "Saffron",

         "What is the primary greenhouse gas emitted by human activities?": "Carbon dioxide (CO2)",
         "What process do plants use to convert sunlight into energy?": "Photosynthesis",
         "What is the term for the variety of life in a particular habitat?": "Biodiversity",
 
         "In what year did the French Revolution begin?": "1789",
         "Who was the first female Prime Minister of the United Kingdom?": "Margaret Thatcher",
         "Which ancient civilization built the Great Pyramid of Giza?": "The Egyptians",
         
         "Which of the planets in our galaxy has the most massive size?":"Jupiter",
         "What is the variable type that only holds true or false data?":"A data type that has one of two possible values: true or false.",
          "Can you tell me the primary component used to make standard guacamole?":"Avocado",
          "How do we define the total variety of different species in a specific ecosystem?":"Biodiversity",
          "Which group of ancient people was responsible for constructing the Giza pyramids?":"The Egyptians"
          
         
          
          
         
}
    



In [30]:
docs=[]

for question, answer in qa_dataset.items():
    d=Document(page_content=question, metadata={"answer": answer})
    docs.append(d)

In [31]:
#  initialize FAISS vector store
vector_store = FAISS.from_documents(documents=docs, embedding=embedding_model)


## Retrivers

1. Vector store as retriever
2. Similarity Score Threshold retriever 
3. MMR(Maximal Marginal Relevance) retriever
4. Multi-Query retriever
5. Contextual Compression Retriever
6. Data source specific retriever [Wikipedia retriver, Arxiv retriver, vectrostore retriver]
7. ParentDocumentRetriever
8. MultiVectorretriever
9. SelfQueryRetriever
10. EnsembleRetriever
11. TimeWeightedRetriever

#### Q) If vector database  only provide method to get relevent documents then why retrivers required?
- Yes vector store provide one method to get relevant documents, but retrivers add more functionality to it, hence retrivers also required 

### 1. Vector store as retriver

- Uses cosine similarity to find relevant documents

In [33]:
# retriver
retriever=vector_store.as_retriever()


# sample query
user_question="What is the largest planet in our solar system?"

# retrieve top 3 similar questions
retrieved_docs = retriever.invoke(user_question,k=3)
for doc in retrieved_docs:
    print(f"Question: {doc.page_content}")
    print(f"Answer: {doc.metadata['answer']}\n")

Question: What is the largest planet in our solar system?
Answer: Jupiter

Question: Which of the planets in our galaxy has the most massive size?
Answer: Jupiter

Question: Which planet is known as the Red Planet?
Answer: Mars



In [34]:
#  get result along with similarity score
results = vector_store.similarity_search_with_relevance_scores(user_question, k=3)

for doc, score in results:
    print(f"Similarity Score: {score:.4f}")
    print(f"Question: {doc.page_content}\n")
    print(f"Answer: {doc.metadata['answer']}\n")
    print("-----"*10)

Similarity Score: 1.0000
Question: What is the largest planet in our solar system?

Answer: Jupiter

--------------------------------------------------
Similarity Score: 0.6320
Question: Which of the planets in our galaxy has the most massive size?

Answer: Jupiter

--------------------------------------------------
Similarity Score: 0.4032
Question: Which planet is known as the Red Planet?

Answer: Mars

--------------------------------------------------


### 2. Similarity Score Threshold retriever 

**Working Principle:** Same as vector store as retriver but here we can add similarity threshold, the documnets with lesser similarity than given threshold will not fetched

**Advantaged:** It takes only highly related documents only

In [36]:
# retriver
retriever_with_threshold = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.5}  # u can play with threshold value
)
# Only returns matches >80% similar


# sample query
user_question="What is the largest planet in our solar system?"

#  get result along with similarity score
results = retriever_with_threshold.invoke(user_question)

for doc in results:
    # print(f"Similarity Score: {score:.4f}")
    print(f"Question: {doc.page_content}\n")
    print(f"Answer: {doc.metadata['answer']}\n")
    print("-----"*10)

Question: What is the largest planet in our solar system?

Answer: Jupiter

--------------------------------------------------
Question: Which of the planets in our galaxy has the most massive size?

Answer: Jupiter

--------------------------------------------------


### 3. MMR(Maximal Marginal Relevance) 
**Working Principles:**
- Designed to reduce redundancy in the retrieved result while Maintains high relevancy to the query
- Picks most relevant document first
- Then select next most relevant document and least relevant to the already selected document

**Advantages:**
- Reduce the repeating of same information
- Enhance the the diversity in the answer

#### MMR Search Keyword Definitions

*   **`k: 3` (The Final Count)**
    *   This is the total number of documents that will ultimately be returned to you. Even if the AI finds 50 relevant items, it will output only the **top 3** most diverse and relevant ones.

*   **`fetch_k: 10` (The Initial Pool)**
    *   The retriever first grabs the **10** most similar documents based purely on cosine similarity.
    *   **Why?** MMR needs a "candidate pool" to evaluate and compare for diversity.
    *   **Rule of Thumb:** Always set `fetch_k` significantly higher than `k` (usually 3x or 4x).

*   **`lambda_mult: 0.5` (The Diversity Slider)**
    *   This is the "magic" number that balances **Similarity vs. Diversity** on a scale of **0.0 to 1.0**:
        *   **1.0 (Pure Similarity):** Acts exactly like a standard similarity search; it does not prioritize diversity.
        *   **0.5 (Balanced):** The default setting. It looks for documents that are highly relevant but also "different" from one another.
        *   **0.0 (Pure Diversity):** Prioritizes results that are as unique as possible from each other, even if they aren't the absolute closest matches to the query.

In [38]:
mmr_retriever = vector_store.as_retriever(
    search_type="mmr", 
    search_kwargs={"k": 3, "fetch_k": 10, "lambda_mult": 0.1}
)
# fetch_k=10 (wide net), k=3 (final diverse results)


# sample query
user_question="What is the largest planet in our solar system?"

#  get result
results = mmr_retriever.invoke(user_question)

for doc in results:
    # print(f"Similarity Score: {score:.4f}")
    print(f"Question: {doc.page_content}\n")
    print(f"Answer: {doc.metadata['answer']}\n")
    print("-----"*10)


Question: What is the largest planet in our solar system?

Answer: Jupiter

--------------------------------------------------
Question: Which ancient civilization built the Great Pyramid of Giza?

Answer: The Egyptians

--------------------------------------------------
Question: Who was the first female Prime Minister of the United Kingdom?

Answer: Margaret Thatcher

--------------------------------------------------


### 4. Multi-Query retriever

**Working principle:**
- For give query it generates some more relevent querries using llm
- For each generated querries, it fetch the relevent document from the vector store
- out of all featched documents fetch the top k most relevant querries

**Advantages:**
- Increase the recall of the query

**Example:**

User Query: "How do neural networks work?"

↓ LLM Query Generator

3 Variations:
1. "Explain neural network functionality"
2. "Mechanism of neural networks"  
3. "How do NNs process information?"

↓ Similarity Search (ALL 3 queries)

↓ Combine + De-duplicate Results

↓ Return diverse, comprehensive documents

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
from langchain_core.documents import Document

d:\Learning\RAG\RAG_ENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. Setup vector store with diverse docs
docs = [
    Document(page_content="Neural networks process data through layers of interconnected nodes"),
    Document(page_content="Deep learning models use backpropagation for training"),
    Document(page_content="Transformers revolutionized sequence modeling"),
    Document(page_content="CNNs excel at image recognition tasks")
]

In [3]:
vectorstore = FAISS.from_documents(docs, OpenAIEmbeddings())

In [5]:


llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# ✅ CORRECT for latest LangChain
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=base_retriever,
    llm=llm
)


In [6]:
# 3. Test it
user_query = "How do neural networks work?"

In [7]:

results = multi_query_retriever.invoke(user_query)

print("=== MULTIQUERY RESULTS ===")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content}")


=== MULTIQUERY RESULTS ===
1. Neural networks process data through layers of interconnected nodes
2. Deep learning models use backpropagation for training


### 5. Contextual Compression Retriever
**Working Principle:**
- Retreive document as same as similarity rearch
- But while returning the result , it remove the unwanted content in each document

**Advantage:**
- Only related content will be available for llm
- Improve the accuracy of generation

In [11]:
from langchain_classic.retrievers import ContextualCompressionRetriever

In [12]:
docs = [
    Document(page_content="Neural networks process data through layers of interconnected nodes. Tiger is that national animal of india"),
    Document(page_content="Deep learning models use backpropagation for training. Deer run very fast"),
    Document(page_content="Transformers revolutionized sequence modeling. I love india"),
    Document(page_content="CNNs excel at image recognition tasks. Elephant is the largest land animal")
]

vectorstore = FAISS.from_documents(docs, OpenAIEmbeddings())

In [16]:
from langchain_classic.retrievers.document_compressors import LLMChainExtractor

In [40]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
compressor = LLMChainExtractor.from_llm(llm)  # LLM decides what to keep

base_retriever = vectorstore.as_retriever()

In [41]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever,
    k=3
)

In [44]:
retrieved_docs=compression_retriever.invoke("which network used in image recognisition")

In [45]:
retrieved_docs

[Document(metadata={}, page_content='CNNs excel at image recognition tasks.')]

In [35]:
# Note: There are many techniques available for compressor 